In [2]:
import yaml
import argparse
import numpy as np
import sys
from bayesgm.models import CausalBGM, BGM, MNISTBGM
from bayesgm.datasets import (
    GMM_indep_sampler, 
    Swiss_roll_sampler, 
    simulate_regression, 
    simulate_low_rank_data, 
    simulate_heteroskedastic_data, 
    simulate_z_hetero
)

In [1]:
1

1

In [3]:
import tensorflow as tf
print("TensorFlow Version:", tf.__version__)
#tf.config.set_visible_devices([], 'GPU')
# Check if a GPU is available
gpu_available = tf.config.list_physical_devices('GPU')
print(gpu_available)
#tf.config.experimental.set_memory_growth(gpu_available[0], True)

import sys,time,os
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
#!ml py-tensorflow/2.10.0_py39
import pandas as pd
import seaborn as sns
from scipy.stats import pearsonr, spearmanr
from bayesgm.datasets import simulate_z_hetero
from sklearn.model_selection import train_test_split

TensorFlow Version: 2.10.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
config = 'configs/Sim_heteroskedastic.yaml'
with open(config, 'r') as f:
    params = yaml.safe_load(f)
# E, B = 200, 50000
# params['kl_weight'] = 0.00005
# params['lr_theta'] = 0.005
# params['lr_z'] = 0.005
# params['alpha'] = 0
# params['g_units'] = [128, 128, 128]
# params['e_units'] = [128, 128, 128]
# params['z_dim'] = 10
# params['x_dim'] = 100
# params['use_bnn'] = False

In [5]:
from bayesgm.datasets import simulate_z_hetero
X,Y = simulate_z_hetero(n=20000, k=params['z_dim'], d=params['x_dim']-1)
np.random.seed(123)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=123)
data = np.c_[X_train, Y_train].astype('float32')
X_test = X_test.astype('float32')
Y_test = Y_test.astype('float32')

In [ ]:
model = BGM(params=params, random_seed=None)
model.fit(data=data, epochs=200, epochs_per_eval=10, use_egm_init=True, egm_n_iter=50000, egm_batches_per_eval=500, verbose=1)

Metal device set to: Apple M4

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2026-02-12 23:09:09.010118: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2026-02-12 23:09:09.010797: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


EGM Initialization Starts ...


2026-02-12 23:09:10.624809: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2026-02-12 23:09:10.634043: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [ ]:
data = np.hstack([X_test, np.full((X_test.shape[0], 1), np.nan)])
data_x_pred, pred_interval = model.predict(data=data, 
                                            alpha=0.05, 
                                            return_samples=True,
                                            bs=100,
                                            n_mcmc=5000, 
                                            burn_in=5000, 
                                            step_size=0.01, 
                                            num_leapfrog_steps=10, 
                                            seed=42)
print(data_x_pred.shape, pred_interval.shape)

2026-02-12 02:08:27.745940: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


TFP MCMC Acceptance Rate: 0.7447


2026-02-12 02:32:02.395928: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


(5000, 2000, 100) (2000, 1, 2)


In [1]:
def simulate_z_hetero(n=20000, k=2, d=5, seed=42):
    np.random.seed(seed)

    Z = np.random.randn(n, k)

    # Low-rank X from latent Z
    A = np.random.randn(d, k)
    X = 0.2*Z @ A.T + 0.1 * np.random.randn(n, d)

    # Define nonlinear mean and heteroskedastic std
    w = np.random.randn(k)
    u = np.random.randn(k)

    
    mean_Y = np.sin(Z @ w)
    std_Y = 0.1 + 0.5 * 1 / (1 + np.exp(-(Z @ u)))

    Y = mean_Y + std_Y * np.random.randn(n)
    return X, Y, Z, A, w, u

def generative_function(x_test, A, w, u, N):
    """
    Given x_test of shape (n, d), sample size N, and parameters A, w, u,
    return sampled z from P(z|x) with shape (N, n, k) and corresponding y with shape (N, n)
    """
    n, d = x_test.shape
    k = A.shape[1]

    # Constants from the generative model
    sigma_x = 0.01  # noise variance in X
    scale = 0.2     # scale factor in X generation

    # Posterior covariance: (I_k + scale^2 / sigma_x * A^T A)^-1
    precision = np.eye(k) + (scale**2 / sigma_x) * (A.T @ A)
    cov_z_given_x = np.linalg.inv(precision)  # shape (k, k)

    # Posterior mean: cov_z_given_x @ (scale / sigma_x) * A^T x_test.T
    mean_z_given_x = (cov_z_given_x @ ((scale / sigma_x) * A.T @ x_test.T)).T  # shape (n, k)

    # Sample Z from P(z|x)
    Z_samples = np.random.randn(N, n, k) @ np.linalg.cholesky(cov_z_given_x).T  # shape (N, n, k)
    Z_samples += mean_z_given_x[None, :, :]  # broadcast mean to shape (N, n, k)

    # Compute mean and std for Y
    mean_Y = np.sin(np.einsum('nik,k->ni', Z_samples, w))  # shape (N, n)
    std_Y = 0.1 + 0.5 / (1 + np.exp(-np.einsum('nik,k->ni', Z_samples, u)))  # shape (N, n)

    # Sample Y from N(mean_Y, std_Y^2)
    Y_samples = mean_Y + std_Y * np.random.randn(N, n)

    return Z_samples, Y_samples

def snr_from_latents(Z, w, u, idx=None):
    """
    SNR = Var( sin(w^T Z) ) / E[ (0.1 + 0.5 * sigmoid(u^T Z))^2 ]
    Optionally restrict to a subset with idx (e.g., test indices).
    """
    if idx is not None:
        Z = Z[idx]
    proj_w = Z @ w
    proj_u = Z @ u
    mean_Y = np.sin(proj_w)
    std_Y  = 0.1 + 0.5 / (1.0 + np.exp(-proj_u))
    num = np.var(mean_Y, ddof=1)
    den = np.mean(std_Y**2)
    return num / den

In [2]:
z_dim, x_dim = 10, 100
X,Y,Z,A,w,u = simulate_z_hetero(n=20000, k=z_dim, d=x_dim-1)
snr_all  = snr_from_latents(Z, w, u)
print('snr_all:',snr_all)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=123)
Z_samples, Y_samples = generative_function(X_test, A, w, u, 5000)
alpha=0.05
length_gt = np.quantile(Y_samples, 1-alpha/2, axis=0)-np.quantile(Y_samples, alpha/2, axis=0)
print(np.mean(length_gt))

NameError: name 'np' is not defined

In [ ]:
corr, _ = pearsonr(np.mean(data_x_pred,axis=0)[:,-1], Y_test)
upper = pred_interval[:,0,1]
lower = pred_interval[:,0,0]
bgm_length = upper - lower
covered = (Y_test >= lower) & (Y_test <= upper)

print(f'interval length PCC: {pearsonr(bgm_length, length_gt)[0]:.4f}')
print(f'interval length Spearman: {spearmanr(bgm_length, length_gt)[0]:.4f}')
print(f'Average coverage rate: {np.mean(covered):.4f}')
print(f'Average length: {np.mean(bgm_length):.4f}')

# CausalBGM

In [6]:
import yaml
from bayesgm.datasets import (
    Sim_Hirano_Imbens_sampler, 
    Sim_Sun_sampler, 
    Sim_Colangelo_sampler, 
    Semi_Twins_sampler, 
    Semi_acic_sampler
)
from bayesgm.models import CausalBGM

In [8]:
x,y,v = Sim_Hirano_Imbens_sampler(N=20000, v_dim=200).load_all()

with open('configs/Sim_Hirano_Imbens.yaml', 'r') as f:
    params = yaml.safe_load(f)
# Instantiate a CausalBGM model
model = CausalBGM(params=params, random_seed=None)

# Train the CausalBGM model with an iterative updating algorithm
model.fit(data=(x,y,v), epochs=100, epochs_per_eval=10, use_egm_init=True, egm_n_iter=30000, egm_batches_per_eval=500, verbose=1)

# Make predictions using the trained CausalBGM model
causal_pre, pos_intervals = model.predict(data=(x,y,v), alpha=0.01, n_mcmc=3000, x_values=np.linspace(0,3,20), q_sd=1.0)

Metal device set to: Apple M4

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2026-02-11 19:10:09.197503: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2026-02-11 19:10:09.199248: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/ql339/anaconda3/envs/bgm/lib/python3.9/site-packages/tensorflow_probability/python/layers/util.py:95: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  loc = add_variable_fn(
/Users/ql339/anaconda3/envs/bgm/lib/python3.9/site-packages/tensorflow_probability/python/layers/util.py:105: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use th

EGM Initialization Starts ...


2026-02-11 19:10:12.637961: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2026-02-11 19:10:12.646348: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2026-02-11 19:10:18.499264: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


EGM Initialization Iter [0] : e_loss_adv [0.0754], l2_loss_v [1.0677], l2_loss_z [0.9241], l2_loss_x [13.4237], l2_loss_y [16.1360], g_e_loss [31.6270], dz_loss [0.1932], d_loss [0.2768]


...

Epoch 90/100: 100%|██████████| 625/625 [00:34<00:00, 18.06batch/s, loss_px_z: [0.7146], loss_mse_x: [0.5803], loss_py_z: [0.8618], loss_mse_y: [1.9805], loss_pv_z: [99.1276], loss_mse_v: [0.9812], loss_postrior_z: [105.8549]]  


Epoch [90/100]: MSE_x: 2.3142, MSE_y: 1.2739, MSE_v: 0.9750



Epoch 100/100: 100%|██████████| 625/625 [00:35<00:00, 17.79batch/s, loss_px_z: [0.5351], loss_mse_x: [0.6239], loss_py_z: [0.5814], loss_mse_y: [1.0637], loss_pv_z: [101.2959], loss_mse_v: [1.0031], loss_postrior_z: [105.5140]] 


Epoch [100/100]: MSE_x: 2.2173, MSE_y: 1.2911, MSE_v: 0.9750



NameError: name 'np' is not defined

In [ ]:
import numpy as np
causal_pre, pos_intervals = model.predict(data=(x,y,v), alpha=0.01, n_mcmc=3000, x_values=np.linspace(0,3,20), q_sd=1.0)

MCMC Latent Variable Sampling ...


2026-02-11 21:40:29.268803: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Final MCMC Acceptance Rate: 0.1205


2026-02-11 21:50:23.069387: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Final MCMC Acceptance Rate: 0.1202
